# DO UPVOTE GUYS (THIS WAS MY FIRST CODE TBH)!

In [ ]:
# import the data
import pandas as pd

training = pd.read_csv('/kaggle/input/llm-classification-finetuning/train.csv')
test = pd.read_csv('/kaggle/input/llm-classification-finetuning/test.csv')

training['train_test'] = 1
test['train_test'] = 0
all_data = pd.concat([training,test])

print("Import Data Complete")

In [ ]:
training.head(10)
training.tail(10)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Separate vectorizers for each field
vectorizer_prompt = TfidfVectorizer(max_features=150)
vectorizer_response_a = TfidfVectorizer(max_features=150)
vectorizer_response_b = TfidfVectorizer(max_features=150)

# Fit on training data
temp_prompt = vectorizer_prompt.fit_transform(training["prompt"])
temp_response_a = vectorizer_response_a.fit_transform(training["response_a"])
temp_response_b = vectorizer_response_b.fit_transform(training["response_b"])

print("TF-IDF Features for Prompt:", vectorizer_prompt.get_feature_names_out())
print("TF-IDF Features for Response A:", vectorizer_response_a.get_feature_names_out())
print("TF-IDF Features for Response B:", vectorizer_response_b.get_feature_names_out())

print("Number of elements for the vectorizer representation for 'prompt':", temp_prompt.shape)
print("Number of elements for the vectorizer representation for 'response a':", temp_response_a.shape)
print("Number of elements for the vectorizer representation for 'response b':", temp_response_b.shape)

#selecting the prediction target
def get_winner(row):
    if row['winner_model_a'] == 1:
        return 0  # response_a wins
    elif row['winner_model_b'] == 1:
        return 1  # response_b wins
    elif row['winner_tie'] == 1:
        return 2  # tie
    else:
        return -1  # unknown or missing

training['winner'] = training.apply(get_winner, axis=1)
train_y = training["winner"].values

#choosing "features"
train_X = np.concatenate((temp_prompt.toarray(), temp_response_a.toarray(), temp_response_b.toarray()), axis=1)

print("Selecting The Prediction Target and Choosing Features Complete")
#use Logistic Regression
from sklearn.linear_model import LogisticRegression
from datetime import datetime

#record start time to calculate the execution time
start = datetime.now()

#Logistic Regression
model = LogisticRegression(max_iter=500, multi_class='multinomial', solver='saga') #For large datasets the “saga” solver is usually faster [scikit-learn documentation]
model.fit(train_X, train_y)

#record end time
end = datetime.now()
 
#calculate the execution time
execution_time = (end - start).total_seconds() / 60
print(f"The time of execution is: {execution_time} minutes")


print("Model Training Complete")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score

#split into validation and training data
train_X_train, train_X_val, train_y_train, train_y_val = train_test_split(train_X, train_y, test_size=0.2, random_state=42)

#record start time to calculate the execution time
start = datetime.now()

#think about results - comparing predictions (value_y_predict) to the actual winner model (train_y_val)
value_y_predict = model.predict(train_X_val)
print('Model winner prediction', value_y_predict)
print('Model winner real value', train_y_val)

value_y_probabilities = model.predict_proba(train_X_val)
print('Model winner prediction, probability', value_y_probabilities) #winner model a | winner model b | winner tie

#confusion matrix
cm = confusion_matrix(train_y_val, value_y_predict)
print("Confusion Matrix:\n", cm)

#model accuracy
score = model.score(train_X_val, train_y_val)
print('Model Accuracy Score', score)

#macro and micro averaged Precision and Recall
macro_precision = precision_score(train_y_val, value_y_predict, average='macro') #calculate precision for all classes individually and then average them
macro_recall = recall_score(train_y_val, value_y_predict, average='macro')
micro_precision = precision_score(train_y_val, value_y_predict, average='micro') #calculate class wise true positive and false positive and then use that to calculate overall precision
micro_recall = recall_score(train_y_val, value_y_predict, average='micro')
print("Macro Precision:", macro_precision)
print("Macro Recall:", macro_recall)
print("Micro Precision:", micro_precision)
print("Micro Recall:", micro_recall)

#record end time
end = datetime.now()
 
#calculate the execution time
execution_time = (end - start).total_seconds()
print(f"The time of execution is: {execution_time} seconds")

In [ ]:
#model log loss - https://www.kaggle.com/competitions/llm-classification-finetuning/discussion/552103
from sklearn.metrics import log_loss

model_log_loss = log_loss(train_y_val, value_y_probabilities)

print('Model Log loss:', model_log_loss) 

# Number of classes = 3 : Logloss = - log(1/3) = 1.10
# Model Log loss: 1.05, model prediction is considered good for this project

In [ ]:
#model prediction
# transform test data using the trained vectorizers
temp_test_prompt = vectorizer_prompt.transform(test["prompt"])
temp_test_response_a = vectorizer_response_a.transform(test["response_a"])
temp_test_response_b = vectorizer_response_b.transform(test["response_b"])

test_X = np.concatenate((temp_test_prompt.toarray(), temp_test_response_a.toarray(), temp_test_response_b.toarray()), axis=1)
value_test_y_probabilities = model.predict_proba(test_X)
print('Model winner prediction, probability', value_test_y_probabilities) #winner model a | winner model b | winner tie

In [ ]:
output = pd.DataFrame({'id': test.id,
                        'winner_model_a': value_test_y_probabilities[:, 0],
                        'winner_model_b': value_test_y_probabilities[:, 1],
                        'winner_tie': value_test_y_probabilities[:, 2]})
output.to_csv('submission.csv', index=False)